In [1]:
import numpy as np
import xarray as xr

##--##--##--##--  读取 ERA5 2米气温数据  --##--##--##--##
ds0 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m.2021-05.daily.nc")
t0 = ds0.t2m.loc[:,90:0,:]
print(np.shape(t0))
t = np.zeros((30, 153, np.shape(t0)[1],np.shape(t0)[2]))

for iyear in range(30):
  print('iyear = ',iyear+1981)             ##   1981-2010, for cold extreme threshold
  ds1 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m."+str(iyear+1981)+"-05.daily.nc")
  t[iyear,0:31,:,:] = ds1.t2m.loc[:,90:0,:]
  ds2 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m."+str(iyear+1981)+"-06.daily.nc")
  t[iyear,31:61,:,:] = ds2.t2m.loc[:,90:0,:]
  ds3 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m."+str(iyear+1981)+"-07.daily.nc")
  t[iyear,61:92,:,:] = ds3.t2m.loc[:,90:0,:]
  ds4 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m."+str(iyear+1981)+"-08.daily.nc")
  t[iyear,92:123,:,:] = ds4.t2m.loc[:,90:0,:]
  ds5 = xr.open_dataset("/home/ys17-23/Extension/ERA5/ERA5-daily/surface/t2m/t2m."+str(iyear+1981)+"-09.daily.nc")
  t[iyear,123:153,:,:] = ds5.t2m.loc[:,90:0,:]
  ds1.close(); ds2.close(); ds3.close(); ds4.close(); ds5.close()

/home/soft/anaconda3/lib/python3.8/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.21.0 or higher is recommended. You are running version 2.17.0
  warnings.warn(


(31, 91, 360)
iyear =  1981
iyear =  1982
iyear =  1983
iyear =  1984
iyear =  1985
iyear =  1986
iyear =  1987
iyear =  1988
iyear =  1989
iyear =  1990
iyear =  1991
iyear =  1992
iyear =  1993
iyear =  1994
iyear =  1995
iyear =  1996
iyear =  1997
iyear =  1998
iyear =  1999
iyear =  2000
iyear =  2001
iyear =  2002
iyear =  2003
iyear =  2004
iyear =  2005
iyear =  2006
iyear =  2007
iyear =  2008
iyear =  2009
iyear =  2010


In [2]:
##--##--##--##--  计算夏天15天滑动90%分位阈值  --##--##--##--##
t_15days = np.zeros((30*15, 92, np.shape(t0)[1],np.shape(t0)[2]))
print(t_15days.size * t_15days.itemsize / 1073741824)  ## 3.57G
for iday in range(92):
  if iday%30==0: print('iday = ',iday)
  for lag in range(15):
    t_15days[30*lag:30*lag+30,iday,:,:] = t[:,31+iday+lag-7,:,:]

t_th10 = np.percentile(t_15days, 10, axis=0)
np.shape(t_th10)

10.104954242706299
iday =  0


iday =  30
iday =  60
iday =  90


(92, 91, 360)

In [3]:
##--##--##--##--  写进nc文件内  --##--##--##--##
air_th10= xr.DataArray(data=t_th10.data, dims=['day','lat','lon'],
                      coords={'day':np.arange(1,93,1),
                              'lat':t0.latitude.data, 'lon':t0.longitude.data})
ds2 = xr.Dataset(data_vars=dict(air_th10=air_th10))
ds2.to_netcdf("/home/ys17-23/cai_fy/data4_heatwaves/NCC_1980_2010/1_threshold/era5_Tave_th10_JJA_1981_2010.nc")
ds2.close()

ds3 = xr.open_dataset("/home/ys17-23/cai_fy/data4_heatwaves/NCC_1980_2010/1_threshold/era5_Tave_th10_JJA_1981_2010.nc")
ds3

<xarray.Dataset>
Dimensions:   (day: 92, lat: 91, lon: 360)
Coordinates:
  * day       (day) int64 1 2 3 4 5 6 7 8 9 10 ... 83 84 85 86 87 88 89 90 91 92
  * lat       (lat) float32 90.0 89.0 88.0 87.0 86.0 ... 4.0 3.0 2.0 1.0 0.0
  * lon       (lon) float32 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
Data variables:
    air_th10  (day, lat, lon) float64 ...

In [4]:
# import numpy as np
# print(t_True_3D.size * t_True_3D.itemsize/1073741824)